In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("IntroToPySpark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 10:14:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [2]:
data = [("John", "Doe", 29), ("Jane", "Smith", 34), ("Sam", "Brown", 23)]
columns = ["first_name", "last_name", "age"]

df = spark.createDataFrame(data, columns)
df.show()

+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      John|      Doe| 29|
|      Jane|    Smith| 34|
|       Sam|    Brown| 23|
+----------+---------+---+



In [5]:
df_from_csv = spark.read.csv("../csv/cereal.csv", header=True, inferSchema=True)
df_from_csv.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [6]:
# Selecting specific columns
df.select("first_name", "age").show()

# Filtering data
df.filter(df["age"] > 25).show()

# Grouping and Aggregating
df.groupBy("age").count().show()

+----------+---+
|first_name|age|
+----------+---+
|      John| 29|
|      Jane| 34|
|       Sam| 23|
+----------+---+

+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      John|      Doe| 29|
|      Jane|    Smith| 34|
+----------+---------+---+

+---+-----+
|age|count|
+---+-----+
| 29|    1|
| 34|    1|
| 23|    1|
+---+-----+



In [7]:
df.createOrReplaceTempView("people")
result = spark.sql("SELECT * FROM people WHERE age > 25")
result.show()

+----------+---------+---+
|first_name|last_name|age|
+----------+---------+---+
|      John|      Doe| 29|
|      Jane|    Smith| 34|
+----------+---------+---+



In [8]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
squared_rdd = rdd.map(lambda x: x*x)
print(squared_rdd.collect())

[1, 4, 9, 16, 25]


In [9]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# Sample data
data = [(Vectors.dense([0.0]), 1.0),
        (Vectors.dense([1.0]), 2.0),
        (Vectors.dense([2.0]), 3.0)]

df = spark.createDataFrame(data, ["features", "label"])

# Linear Regression model
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(df)

# Print the coefficients
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

23/12/12 10:18:35 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Coefficients: [0.657728271247261]
Intercept: 1.3422717287527388
